<a href="https://colab.research.google.com/github/agemagician/Prot-Transformers/blob/master/Embedding/Advanced/Albert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Extracting protein sequences' features using ProtAlbert pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 675kB 3.2MB/s 
     |████████████████████████████████| 3.8MB 7.1MB/s 
     |████████████████████████████████| 1.1MB 49.5MB/s 
     |████████████████████████████████| 890kB 45.2MB/s 


In [2]:
import torch
from transformers import AlbertModel, AlbertTokenizer
import re
import os
import requests
from tqdm.auto import tqdm

<b>2. Set the url location of ProtAlbert and the vocabulary file<b>

In [3]:
modelUrl = 'https://www.dropbox.com/s/gtajtmege43ec7k/pytorch_model.bin?dl=1'
configUrl = 'https://www.dropbox.com/s/me7zsqrnpiz043v/config.json?dl=1'
tokenizerUrl = 'https://www.dropbox.com/s/60mg00r361vth4t/albert_vocab_model.model?dl=1'

<b>3. Download ProtAlbert models and vocabulary files</b>

In [4]:
downloadFolderPath = 'models/ProtAlbert/'

In [5]:
modelFolderPath = downloadFolderPath

modelFilePath = os.path.join(modelFolderPath, 'pytorch_model.bin')

configFilePath = os.path.join(modelFolderPath, 'config.json')

tokenizerFilePath = os.path.join(modelFolderPath, 'spm_model.model')

In [6]:
if not os.path.exists(modelFolderPath):
    os.makedirs(modelFolderPath)

In [7]:
def download_file(url, filename):
  response = requests.get(url, stream=True)
  with tqdm.wrapattr(open(filename, "wb"), "write", miniters=1,
                    total=int(response.headers.get('content-length', 0)),
                    desc=filename) as fout:
      for chunk in response.iter_content(chunk_size=4096):
          fout.write(chunk)

In [8]:
if not os.path.exists(modelFilePath):
    download_file(modelUrl, modelFilePath)

if not os.path.exists(configFilePath):
    download_file(configUrl, configFilePath)

if not os.path.exists(tokenizerFilePath):
    download_file(tokenizerUrl, tokenizerFilePath)

<b>4. Load the vocabulary and ProtAlbert Model<b>

In [9]:
tokenizer = AlbertTokenizer(tokenizerFilePath, do_lower_case=False)

In [10]:
model = AlbertModel.from_pretrained(modelFolderPath)

<b>5. Load the model into the GPU if avilabile and switch to inference mode<b>

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [12]:
model = model.to(device)
model = model.eval()

<b>6. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [13]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [14]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>7. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [15]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, pad_to_max_length=True)

In [16]:
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

<b>8. Extracting sequences' features and load it into the CPU if needed<b>

In [17]:
with torch.no_grad():
    embedding = model(input_ids=input_ids,attention_mask=attention_mask)[0]

In [18]:
embedding = embedding.cpu().numpy()

<b>9. Remove padding ([PAD]) and special tokens ([CLS],[SEP]) that is added by Albert model<b>

In [19]:
features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][1:seq_len-1]
    features.append(seq_emd)

In [20]:
print(features)

[array([[ 0.05041128,  0.01670685, -0.06041763, ...,  0.03879201,
         0.0674293 , -0.02243809],
       [ 0.04481388,  0.02007382, -0.06041713, ...,  0.01449793,
         0.07357242, -0.02993674],
       [ 0.04433505,  0.02113645, -0.0598816 , ...,  0.01485567,
         0.07245695, -0.02932317],
       ...,
       [ 0.04479406,  0.01773301, -0.06211168, ...,  0.01833694,
         0.0708717 , -0.02901287],
       [ 0.03966058,  0.04050258, -0.01892234, ...,  0.02171973,
         0.06361844, -0.04444959],
       [ 0.04467377,  0.02889517, -0.06304097, ...,  0.00830418,
         0.0829025 , -0.02485315]], dtype=float32), array([[-0.03166402,  0.01354338, -0.02900368, ..., -0.11964945,
        -0.11535365,  0.07926654],
       [-0.04238719,  0.01891202, -0.02534003, ..., -0.12101925,
        -0.10938749,  0.09651614],
       [-0.04247086,  0.02128087, -0.02400087, ..., -0.12410779,
        -0.10866657,  0.10307116],
       [-0.04353952,  0.02157599, -0.02257544, ..., -0.12234627,
     